 # TP 6: Apprentissage profond(Deep learning)
 ## TP 6.4  : Optimisation solution naive
 
- Régularisation :  Dropout
- Normalisation 
    - Inputs
    - les autres couches (batch Normalisation: avant ou après l'activation)
- Optimiseurs: 
    - RMSProp, Adam, SGD, 
- Hyperparameter tuning : 
    - learning_rate
    - \#couches
    - \#neurones par couche
    - taille du mini batch 
- ...
# Exercice
Trouver le meilleur modèle pour les données  de "smoking"
Le programme doit tester les différentes configurations (et hyperparamètres).

Visiter le site keras ou tensorflow pour vérifier la maniène d'utiliser ces différents paramètres

PS : comparer aussi avec les modèles classiques : (KNN, Randomforest, ...) 

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import seaborn as sns


### Préparation des données

In [22]:
# Lecture du dataset n permet de limiter le nombre de lignes à lire
# Pour faciliter les tests

def load_data(n):
    data = pd.read_csv('../../data/train.csv')
    return data[0:n]

def select_variables(data):
    data.dropna(axis=0, inplace=True)
    y = data['smoking'] # récupérer la colonne survived et la mettre dans y
    # récuperer le reste des données dans X utiliser la fonction titanic.drop ???, ??? )
    X = data.drop('smoking', axis=1)
    return X,y

#### Split des données

In [23]:
def split_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test  

#### Normalisation

In [24]:
## On peut utiliser une simple normalisation (x-mu)/ecart type)
def normaliser(X_train, X_test):
    mean = X_train.mean()
    std  = X_train.std()
    X_train = (X_train - mean) / std
    X_test  = (X_test  - mean) / std

    return X_train, X_test 

### Définition du modèle 
avec prise d'un nombre de couches et nombre de neurones variables 

In [25]:
def build_model(nb_layers, nb_units):
    m = X_train.shape[1]
    model = tf.keras.Sequential ()
    model.add(tf.keras.layers.Input(m, name="InputLayer"))
    for l in range(nb_layers):
        model.add(tf.keras.layers.Dense(nb_units,name='layer'+str(l)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid',name='output'))
    return model

In [26]:
#Compiler le modèle
#Optimiserer: SGD, AdamW, adadelta, ...
def compiler(model,optimizer,loss,metrics):
    model.compile(optimizer=optimizer,
                  loss= loss,
                  metrics=[metrics])
    return model



### 4. Entrainement du modèle (Model training)

In [27]:
# Entrainement du modele
def train(model, X_train, y_train,epochs,batch_size):
    history  = model.fit(X_train, 
                     y_train, 
                     epochs=epochs, 
                     batch_size= batch_size, 
                     verbose=False,
                     validation_data = (X_test, y_test))
    return model, history


### 5. Evaluation des performances du modèle 


In [28]:
def evaluer(model, X_test, y_test):
    score = model.evaluate(X_test, y_test, verbose=0)
    return score

#print('Test loss     :', score[0])
#print('Test accuracy :', score[1])


In [29]:
def visualiser_confusion(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # Get class labels
    y_classes = np.argmax(y_pred, axis=-1)

    cm = confusion_matrix(y_test, y_classes)
    #disp= ConfusionMatrixDisplay(confusion_matrix=cm)

    sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
    plt.show()


### Main program

In [30]:
# Préparation ds données
# le -1 du load_data(-1) veut dire on prend toutes les lignes 
data=load_data(-1) 
# sélectionner les variables
X,y = select_variables(data)
X_train, X_test, y_train, y_test = split_data(X,y)
X_train, X_test = normaliser(X_train, X_test)

In [ ]:
history=[]
loss='BinaryCrossentropy'
metrics='accuracy'
nb_layers=[1,2,4,6]
nb_units=[16, 32]
epochs=[10,50, 100]
batch_size=[16, 32]
optimizers = ['Adam', 'RMSprop', 'SGD', 'Adamax']

best_model=0

for l in nb_layers:
    for u in nb_units:
        model = build_model(l,u)
        for optimize in optimizers:
            for ep in epochs:
                for batch in batch_size:
                    model = compiler(model,optimize,loss,metrics)
                    model,history = train(model, X_train, y_train,ep,batch)
                    score = evaluer(model, X_test, y_test)
                    print('nb_layer',l, 'nb_units' ,u)
                    print('optimiseur',optimize,'Test accuracy  epoch   :',ep, 'batch', batch, score[1])
                    if  score[1] > best_model:
                        best_model = score[1]
                        best_param_model = {"#layers":l,"#units": u, "ep":ep, "batch":batch, "optimize":optimize}


nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 10 batch 16 0.7682019472122192
nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 10 batch 32 0.7689868211746216
nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 50 batch 16 0.7689554691314697


#### Afficher les paramètres du meilleur modèle

In [ ]:
best_param_model

In [ ]:
best_param_model["#layers"]

#### Play with the best model

In [ ]:
## sur les données de Text (X_test, y_test)
model = build_model(best_param_model["#layers"],best_param_model["#units"])  
model = compiler(model,best_param_model["optimize"],loss,metrics)
model,history = train(model, X_train, y_train,best_param_model["ep"],best_param_model["batch"])
score = evaluer(model, X_test, y_test)
print('Test loss     :', score[0])
print('Test accuracy :', score[1])


In [ ]:
print('Test loss     :', score[0])
print('Test accuracy :', score[1])